In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

W1005 00:05:57.547605 13236 deprecation.py:323] From <ipython-input-5-a839aeb82f4b>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W1005 00:05:57.549597 13236 deprecation.py:323] From C:\Users\ARPIT\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W1005 00:05:57.680093 13236 deprecation.py:323] From C:\Users\ARPIT\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W1005 00:05:59.954797 13236 deprecation.py:323] From C:\Users\ARPIT\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W1005 00:05:59.963781 13236 deprecation.py:323] From C:\Users\ARPIT\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W1005 00:06:00.798560 13236 deprecation.py:323] From C:\Users\ARPIT\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])

#Add Function to create weight variable
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


#Add function to create bias variable
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

#Add convolution function
def conv2d(x, W):
    return (tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding = "SAME"))

#Add pooling function
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding= "SAME")

#Reshape the data for inputting it to the input layer
x_image= tf.reshape(x, [-1, 28, 28, 1])

#create 1st convolutional and pooling layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image,  W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)


#create 2nd convolutional and pooling layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


#create 3rd convolutional and pooling layer
W_conv3 = weight_variable([5, 5, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

#1st fully connected layer
W_fc1 = weight_variable([128*4*4, 1024])
b_fc1 = bias_variable([1024])

#Reshape the last convolutional layer output to connect with fully connected layer
h_pool2_flat = tf.reshape(h_pool3, [-1, 128*4*4])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+ b_fc1)

#Add dropout layer
rate = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, rate)

#output layer
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2)+ b_fc2

## Training

In [17]:
#Network Base of Computation

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_ , logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#Prediction Measure
correct_prediction = tf.equal(tf.argmax(y_conv , 1),tf.arg_max(y_ , 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction , tf.float32))

#Mention about the number of epochs training has to happen
epochs= 1100

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        #Read a batch of training data
        batch = mnist.train.next_batch(50)
        if i %100 == 0:
            #Add the training accuracy step
            train_accuracy = accuracy.eval(feed_dict = {x: batch[0] , y_:batch[1], rate:1.0})
            print("step %d, training accuracy %g " % (i,train_accuracy))
        train_step.run(feed_dict = {x:batch[0], y_:batch[1], rate:0.5})
    for i in range(epochs):
        #Read a batch of testing data
        batch = mnist.test.next_batch(50)
        if i % 100 == 0:
            #Add the testing accuracy data
            test_accuracy = accuracy.eval(feed_dict = {x: batch[0], y_:batch[1], rate:1.0})
            print('test accuracy %g'% test_accuracy)

step 0, training accuracy 0.08 
step 100, training accuracy 0.78 
step 200, training accuracy 0.86 
step 300, training accuracy 0.94 
step 400, training accuracy 0.9 
step 500, training accuracy 0.92 
step 600, training accuracy 0.9 
step 700, training accuracy 0.98 
step 800, training accuracy 0.98 
step 900, training accuracy 0.98 
step 1000, training accuracy 1 
test accuracy 0.92
test accuracy 0.94
test accuracy 0.96
test accuracy 0.98
test accuracy 0.98
test accuracy 0.96
test accuracy 0.98
test accuracy 0.94
test accuracy 0.92
test accuracy 1
test accuracy 1
